## Spelling Bee - Sequence Modeling using LSTM - Keras

In the notebook we will explore how to correctly spell words using their phonetic pronunciations, i.e. we will take the pronunciation of a word, and then try to spell it correctly.

We will be using Recurrent Neural Networks for this task (in the following notebook, we will add attention modeling to it).

## Imports

In [100]:
from __future__ import division,print_function
from PIL import Image
import gc,re

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Embedding, Reshape, RepeatVector, \
                         UpSampling2D, Flatten, Dropout, LSTM, TimeDistributed, Bidirectional
#from keras.layers.merge import Add
from keras.optimizers import Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from importlib import reload
from keras import backend as K
from keras.datasets import imdb

np.random.seed(7)

from keras.preprocessing.image import load_img, img_to_array

import bcolz
from IPython.display import FileLink
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

from scipy.optimize import fmin_l_bfgs_b
from keras import metrics
from scipy.misc import imsave
import imageio

%matplotlib inline

## Load and Explore Data

The data can be obtained from here : http://www.speech.cs.cmu.edu/cgi-bin/cmudict

In [8]:
PATH = 'data/spellingbee/cmudict-0.7b'

f = open(PATH, encoding='latin1')
lines = []

for line in f:    
    
    #add to lines if starting letter is an alphabet
    if re.match('^[A-Z]', line):
        lines.append(line.strip().split(" "))
        

In [9]:
len(lines)

133779

In [13]:
lines[55]

['ABANTO', '', 'AH0', 'B', 'AE1', 'N', 'T', 'OW0']

Let us look at the data that we have. Each sound/word is followed by its phonetic pronunciation and we have almost 134k such words.

Each phonetic pronunciation is a sequence of phonemes. *Note* that the vowels end in integers. These integers denote the stress levels.

Also, in our list above, we have the word/sound followed by a space and then followed by the corresponding phoneme sequence. Let us split and store word/sound and their pronunciations properly.

In [37]:
lines_data=[]
for line in lines:
    word = line[0]
    phonemes = line[2:len(line)]
    lines_data.append((word,phonemes))


In [38]:
lines_data[55], lines_data[-1]

(('ABANTO', ['AH0', 'B', 'AE1', 'N', 'T', 'OW0']),
 ('ZYWICKI', ['Z', 'IH0', 'W', 'IH1', 'K', 'IY0']))

Now, let us get all the individual phonemes that are present in the dataset

We will also add an **_** to the list which helps in zero-padding.

In [46]:
phonemes = []

for w,ps in lines_data:
    for p in ps:
        phonemes.append(p)

In [47]:
phonemes = ['_'] + sorted(set(phonemes))

In [49]:
phonemes[:10]

['_', 'AA0', 'AA1', 'AA2', 'AE0', 'AE1', 'AE2', 'AH0', 'AH1', 'AH2']

In [50]:
len(phonemes)

70

So we see that we have a total of 70 unique phonemes in our dataset (including the _ that we added for reference to zero-padding.

As with every NLP task, we need to assign indices to letters and then work with those indices. So now we will be creating two dictionaries which will keep track of phonemes to indices and letters to indices.

In [51]:
p2i = {v : k for k,v in enumerate(phonemes)}

In [52]:
p2i

{'AA0': 1,
 'AA1': 2,
 'AA2': 3,
 'AE0': 4,
 'AE1': 5,
 'AE2': 6,
 'AH0': 7,
 'AH1': 8,
 'AH2': 9,
 'AO0': 10,
 'AO1': 11,
 'AO2': 12,
 'AW0': 13,
 'AW1': 14,
 'AW2': 15,
 'AY0': 16,
 'AY1': 17,
 'AY2': 18,
 'B': 19,
 'CH': 20,
 'D': 21,
 'DH': 22,
 'EH0': 23,
 'EH1': 24,
 'EH2': 25,
 'ER0': 26,
 'ER1': 27,
 'ER2': 28,
 'EY0': 29,
 'EY1': 30,
 'EY2': 31,
 'F': 32,
 'G': 33,
 'HH': 34,
 'IH0': 35,
 'IH1': 36,
 'IH2': 37,
 'IY0': 38,
 'IY1': 39,
 'IY2': 40,
 'JH': 41,
 'K': 42,
 'L': 43,
 'M': 44,
 'N': 45,
 'NG': 46,
 'OW0': 47,
 'OW1': 48,
 'OW2': 49,
 'OY0': 50,
 'OY1': 51,
 'OY2': 52,
 'P': 53,
 'R': 54,
 'S': 55,
 'SH': 56,
 'T': 57,
 'TH': 58,
 'UH0': 59,
 'UH1': 60,
 'UH2': 61,
 'UW0': 62,
 'UW1': 63,
 'UW2': 64,
 'V': 65,
 'W': 66,
 'Y': 67,
 'Z': 68,
 'ZH': 69,
 '_': 0}

In [54]:
letters = "_abcdefghijklmnopqrstuvwxyz*"   # This also includes _ which we added earlier and * which will become 
                                           # clear later.
l2i = {v : k for k,v in enumerate(letters)}

In [55]:
l2i

{'*': 27,
 '_': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

Now that we a dictionary which maps phonemes to indices, we can go to our lines_data and replace each phoneme by its index. 

We will also filter our dataset to only include words of length 5 to 15.

In [70]:
min_length_word = 5
max_length_word = 15

pronunciation_dictionary = {}

for w,ps in lines_data:
    
    if (min_length_word <= len(w) <= max_length_word) and re.match("^[A-Z]+$", w):
        temp = []
        for p in ps:
            temp.append(p2i[p])
        pronunciation_dictionary[w.lower()] = temp

In [71]:
len(pronunciation_dictionary)

108006

In [73]:
pronunciation_dictionary['abanto'], lines_data[55]

([7, 19, 5, 45, 57, 47], ('ABANTO', ['AH0', 'B', 'AE1', 'N', 'T', 'OW0']))

Now that we have converted each word into indices of phonemes, we can proceed further. We also see that our dataset has reduced to almost 108k words.

As with most other NLP tasks, we will have an embedding layer as our first layer which would embed each phoneme into some higher dimensional space. As such, we first need to find out the maximum phoneme sequence which is present in our dataset. We will then use that to pad the other sequences so that every phoneme sequence is of the same length.

In [74]:
max_phoneme_sequence = max([len(v) for k,v in pronunciation_dictionary.items()])

In [75]:
max_phoneme_sequence

16

So now we know what there exists at least one word in our dataset whose phoneme sequence is almost 16 phonemes long!

Now time for zero padding. We need to pad phoneme sequences (which have length <16) with zeros. Lets do it ourselves here instead of using the inbuilt Keras method.

Also, our current task involves predicting the spellings using these phonemes. So our labels are words reach of maximum length 15. We also need to pad words of length less than 15 then.

We will start with already zero matrices of the required sizes. For each row, we will fill the first few columns, as required, and the rest will remain zero.

In [85]:
words = np.random.permutation(list(pronunciation_dictionary.keys()))
n = len(words)

input_ = np.zeros((n,max_phoneme_sequence), np.int32)  # inputs are phonemes of max length 16
labels_ = np.zeros((n, max_length_word) , np.int32)     # outputs are words of max length 15

In [86]:
for i, k in enumerate(words):
    
    # fill those indices of input_ which are non-zero 
    for j, p in enumerate(pronunciation_dictionary[k]):
        input_[i][j] = p
    
    # fill those indices of labels_ which are non-zero
    for j,letter in enumerate(k):
        labels_[i][j] = l2i[letter]
    

In [87]:
input_.shape, labels_.shape

((108006, 16), (108006, 15))

In [90]:
input_[55], labels_[55]

(array([54, 36, 55, 42, 35, 46,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32),
 array([18,  9, 19, 11,  9, 14,  7,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32))

In [91]:
from sklearn.model_selection import train_test_split

Now, let us split our data into train and test set.

In [92]:
(input_train, input_test, labels_train, labels_test) = train_test_split(input_, labels_, test_size=0.1)

In [93]:
input_train.shape, labels_train.shape, input_test.shape , labels_test.shape

((97205, 16), (97205, 15), (10801, 16), (10801, 15))

To reiterate, let us check what is the unique number of phonemes, and unique number of letters

In [95]:
n_unique_phonemes = len(phonemes)
n_unique_letters = len(letters)

n_unique_phonemes, n_unique_letters

(70, 28)

## Creating the Model

Now we have set up the stage for creating our model.

References : https://keras.io/layers/recurrent/#lstm

References : https://keras.io/layers/wrappers/

References : https://keras.io/layers/core/#repeatvector

This is just a function which we defined to create cleaner code. Note that return_sequences in LSTM means whether to return the last output in an output of sequences (if set to False) or whether to return the whole output_of_sequences (if set to True)

In [96]:
def get_rnn(return_sequences=True):
    return LSTM(240, dropout=0.1, recurrent_dropout=0.1, return_sequences=return_sequences)

In [107]:
inp = Input(shape=(max_phoneme_sequence,))
x = Embedding(n_unique_phonemes, 120)(inp)

#Encoder

x = Bidirectional(get_rnn())(x)
x = get_rnn(False)(x)

x = RepeatVector(max_length_word)(x)

# Decoder

x = get_rnn()(x)
x = get_rnn()(x)
outp = TimeDistributed(Dense(n_unique_letters, activation='softmax'))(x)

model = Model(inputs=inp, outputs=outp)
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics = ['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 16)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 16, 120)           8400      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 16, 480)           693120    
_________________________________________________________________
lstm_13 (LSTM)               (None, 240)               692160    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 15, 240)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 15, 240)           461760    
_________________________________________________________________
lstm_15 (LSTM)               (None, 15, 240)           461760    
__________

Let us take a look at our model in a little more detail.

* Our inputs will be phoneme sequences of maximum length 16. These are then fed into an embedding layer which converts each of the 70 phonemes into 120 higher dimensional vectors.


* Now, we have a sequences of phoneme embeddings. We need our model to makes some sense of these embedding sequences and convert them into a single distributed representation. This is where RNNs come in. Using RNNs in this scenario makes sense because RNNs can keep state and store memory. This is particularly helpful in our case where the phonemes looked at before might influence the letter which might be generated by the present phoneme.
    
    
    * Why do we have Bidectional? 
    
    * Bidirectional will feed the sequence to an RNN, feed the reverse sequence to another RNN and then concat their results. We do this because in language things that happen later often influence what came before (i.e. in French, "le garcon, la fille" means the boy, the girl; the word for "the" is determined by the gender of the subject, which comes after).
    
* This part of taking the input sequence and then converting it into a vector representation, which captures the inherent characteristics which we need to spell, is called the **ENCODER**.


* The output of the encoder is then fed into a RepeatVector which does nothing but repeat the encoder's output 15 number of times. This is because at each stage, we want the RNN to keep into account the complete word (letters) that it is trying to spell.


* The **DECODER** here is nothing but a bunch of RNNs which take the encoded input and then try to find meaningful insights in that in a way so as to find letters corresponding to them. Its output is fed into a Dense layer with 28 nodes, corresponding to the 28 letters followed by a softmax function.

**NOTE**

We have used 'sparse_categorical_entropy' since we have not done one_hot_encoding for our targets.

In [114]:
model.fit(input_train, np.expand_dims(labels_train,-1), validation_split=0.2, batch_size=128, epochs=2)

Train on 77764 samples, validate on 19441 samples
Epoch 1/2
77764/77764 [==============================] - 114s 1ms/step - loss: 1.5786 - acc: 0.5410 - val_loss: 1.4739 - val_acc: 0.5622
Epoch 2/2
77764/77764 [==============================] - 107s 1ms/step - loss: 1.3731 - acc: 0.5846 - val_loss: 1.2106 - val_acc: 0.6222


In [115]:
model.fit(input_train, np.expand_dims(labels_train,-1), validation_split=0.2, batch_size=1024, epochs=2)

Train on 77764 samples, validate on 19441 samples
Epoch 1/2
77764/77764 [==============================] - 33s 420us/step - loss: 1.1855 - acc: 0.6280 - val_loss: 1.1216 - val_acc: 0.6458
Epoch 2/2
77764/77764 [==============================] - 33s 418us/step - loss: 1.1007 - acc: 0.6517 - val_loss: 1.0182 - val_acc: 0.6775


In [116]:
model.fit(input_train, np.expand_dims(labels_train,-1), validation_split=0.2, batch_size=1024, epochs=5)

Train on 77764 samples, validate on 19441 samples
Epoch 1/5
77764/77764 [==============================] - 33s 420us/step - loss: 1.0124 - acc: 0.6779 - val_loss: 0.9284 - val_acc: 0.7054
Epoch 2/5
77764/77764 [==============================] - 33s 418us/step - loss: 0.9290 - acc: 0.7019 - val_loss: 0.8430 - val_acc: 0.7302
Epoch 3/5
77764/77764 [==============================] - 33s 418us/step - loss: 0.8535 - acc: 0.7234 - val_loss: 0.7719 - val_acc: 0.7512
Epoch 4/5
77764/77764 [==============================] - 33s 418us/step - loss: 0.7936 - acc: 0.7407 - val_loss: 0.7150 - val_acc: 0.7690
Epoch 5/5
77764/77764 [==============================] - 33s 419us/step - loss: 0.7419 - acc: 0.7567 - val_loss: 0.6782 - val_acc: 0.7797


In [117]:
model.fit(input_train, np.expand_dims(labels_train,-1), validation_split=0.2, batch_size=1024, epochs=15)

Train on 77764 samples, validate on 19441 samples
Epoch 1/15
77764/77764 [==============================] - 33s 419us/step - loss: 0.7030 - acc: 0.7690 - val_loss: 0.6448 - val_acc: 0.7898
Epoch 2/15
77764/77764 [==============================] - 33s 419us/step - loss: 0.6680 - acc: 0.7804 - val_loss: 0.6076 - val_acc: 0.7996
Epoch 3/15
77764/77764 [==============================] - 33s 418us/step - loss: 0.6385 - acc: 0.7893 - val_loss: 0.6079 - val_acc: 0.8003
Epoch 4/15
77764/77764 [==============================] - 33s 419us/step - loss: 0.6150 - acc: 0.7965 - val_loss: 0.5596 - val_acc: 0.8162
Epoch 5/15
77764/77764 [==============================] - 33s 418us/step - loss: 0.5874 - acc: 0.8061 - val_loss: 0.5360 - val_acc: 0.8233
Epoch 6/15
77764/77764 [==============================] - 33s 419us/step - loss: 0.5665 - acc: 0.8125 - val_loss: 0.5193 - val_acc: 0.8292
Epoch 7/15
77764/77764 [==============================] - 33s 419us/step - loss: 0.5498 - acc: 0.8176 - val_loss: 0.

The above metric is actually calculating how many letters correct, but we want to know how many of the words we spell correctly.

In [118]:
predictions = model.predict(input_test, batch_size=1024)

In [121]:
predictions.shape, predictions[0].shape

((10801, 15, 28), (15, 28))

Now, for each position in the word, we have the probability of that being one of 28 different characters. Let us just take the character with the maximum probability.

In [122]:
predictions = np.argmax(predictions, axis=2)
predictions.shape, predictions[0].shape

((10801, 15), (15,))

Now we count a prediction as correct only if all the letters in that word match.

In [123]:
count = 0
for real, pred in zip(labels_test, predictions):
    if(all(real == pred)):
        count += 1
acc = count/len(labels_test)
acc

0.2477548375150449

We see that when comparing complete words, the accuracy drops to almost 25%. This is really low. Let us look at a few predictions to see what is happening.

In [126]:
def print_examples(preds):
    print("PRONUNCIATION".ljust(40), "ACTUAL SPELLING".ljust(17), 
          "PREDICTED SPELLING".ljust(17), "IS CORRECT")

    for index in range(20):
        ps = "-".join([phonemes[p] for p in input_test[index]]) 
        real = [letters[l] for l in labels_test[index]] 
        predict = [letters[l] for l in preds[index]]
        print (ps.split("-_")[0].ljust(40), "".join(real).split("_")[0].ljust(17),
            "".join(predict).split("_")[0].ljust(17), str(real == predict))

In [127]:
print_examples(predictions)

PRONUNCIATION                            ACTUAL SPELLING   PREDICTED SPELLING IS CORRECT
L-AY1-T-IY0                              lighty            litty             False
S-W-IH1-SH-ER0                           swisher           swisher           True
S-AE1-L-IY0                              sallee            sally             False
AH0-N-K-W-EH1-S-CH-AH0-N-AH0-B-AH0-L     unquestionable    uncuessiinbble    False
M-IH2-D-W-EH1-S-T                        midwest           midwest           True
K-AA1-N-T-R-AH0-T-EH2-M-P-S              contretemps       contretomps       False
R-IH0-S-EH1-P-SH-AH0-N-IH0-S-T-S         receptionists     recoptiinnits     False
M-AA1-N-AE2-SH                           monash            monash            True
S-UW2-N-Y-IY1                            soonyi            sunii             False
AE1-M-P-L-AH0-F-AY2-ER0-Z                amplifiers        amplifires        False
L-IH1-K-IH0-NG                           licking           licking           True
AO

Some of the words the model is almost able to get correctly. Like 'sallee' vs 'sally' is an understandable mistake. However, the model performs horribly on words such as 'carcinogenic' and 'carsonogeicc'. 

The model is performing horribly on words which have a larger length. 

This can be fixed through attention modeling which we will look at in the next notebook.